In [53]:
# 필수 라이브러리 import
import numpy as np
import json
from scipy.spatial.transform import Rotation
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# util의 모든 함수와 상수 import
from util import *


In [54]:

# 세션 파일 경로
session_file = "/data/ephemeral/home/measure_volume_by_multiview/project/ar_folder/session_1768877751403.jsonl/session_1768877751403.jsonl"

# JSONL 데이터 로드
data = load_jsonl(session_file)
data.sort(key=lambda x: x.get('t_ns', 0))
print(f"총 프레임 수: {len(data)}")

# ===== 앵커 보정 로직 =====
# 첫 프레임 앵커 정보 추출 (A_1)
first_frame_params = data[0]
first_anchor_pos = np.array(first_frame_params.get('anchor_pos', first_frame_params.get('pos', [0.0, 0.0, 0.0])), dtype=np.float32)
first_anchor_quat = np.array(first_frame_params.get('anchor_quat', first_frame_params.get('quat', [0.0, 0.0, 0.0, 1.0])), dtype=np.float32)
first_anchor_rotation = Rotation.from_quat(first_anchor_quat).as_matrix()

print(f"첫 프레임 앵커: pos=({first_anchor_pos[0]:.3f}, {first_anchor_pos[1]:.3f}, {first_anchor_pos[2]:.3f})")

# GL to CV 변환
gl_to_cv = np.array([
    [1,  0,  0],
    [0, -1,  0],
    [0,  0, -1]
], dtype=np.float32)

# 첫 프레임 앵커 변환 행렬 (A_1): OpenGL → OpenCV 좌표계
first_anchor_rotation_cv = gl_to_cv @ first_anchor_rotation @ gl_to_cv.T
first_anchor_pos_cv = gl_to_cv @ first_anchor_pos

A1 = np.eye(4, dtype=np.float32)
A1[:3, :3] = first_anchor_rotation_cv
A1[:3, 3] = first_anchor_pos_cv

print(f"앵커 보정 적용 중...")

# 각 프레임의 카메라 위치와 레이 계산 (앵커 보정 적용)
camera_positions = []
camera_positions_original = []  # 보정 전 위치 (비교용)
ray_origins = []  # 레이 원점들 (카메라 위치, 보정 적용)
ray_directions = []  # 레이 방향 벡터들 (이미지 중심 방향, 보정 적용)

for i, item in enumerate(data):
    # 원본 카메라 pose
    quat = np.array(item['quat'])
    pos = np.array(item['pos'])
    fx = item['fx']
    fy = item['fy']
    cx = item['cx']
    cy = item['cy']
    
    camera_positions_original.append(pos.copy())
    
    # 현재 프레임 앵커 pose (A_i)
    current_anchor_pos = np.array(item.get('anchor_pos', item.get('pos', [0.0, 0.0, 0.0])), dtype=np.float32)
    current_anchor_quat = np.array(item.get('anchor_quat', item.get('quat', [0.0, 0.0, 0.0, 1.0])), dtype=np.float32)
    current_anchor_rotation = Rotation.from_quat(current_anchor_quat).as_matrix()
    
    # A_i 행렬 (OpenCV 좌표계)
    current_anchor_rotation_cv = gl_to_cv @ current_anchor_rotation @ gl_to_cv.T
    current_anchor_pos_cv = gl_to_cv @ current_anchor_pos
    
    Ai = np.eye(4, dtype=np.float32)
    Ai[:3, :3] = current_anchor_rotation_cv
    Ai[:3, 3] = current_anchor_pos_cv
    
    # 보정 변환: C_i = A_1 * A_i^(-1)
    # 이 변환은 "현재 프레임의 월드 좌표계"를 "첫 프레임의 월드 좌표계"로 되돌림
    Ai_inv = np.linalg.inv(Ai)
    Ci = A1 @ Ai_inv
    
    # 카메라 pose를 OpenCV 좌표계로 변환
    R_c2w_gl = Rotation.from_quat(quat).as_matrix()
    R_c2w_cv = gl_to_cv @ R_c2w_gl @ gl_to_cv.T
    pos_cv = gl_to_cv @ pos
    
    # 원본 카메라 pose 행렬 (P_i): c2w (카메라 → 월드)
    Pi = np.eye(4, dtype=np.float32)
    Pi[:3, :3] = R_c2w_cv
    Pi[:3, 3] = pos_cv
    
    # 보정된 카메라 pose: P_i' = C_i * P_i
    Pi_corrected = Ci @ Pi
    
    # 보정된 pose에서 위치와 방향 추출
    pos_corrected_cv = Pi_corrected[:3, 3]
    R_corrected_cv = Pi_corrected[:3, :3]
    
    # OpenCV → OpenGL 좌표계 변환
    cv_to_gl = np.array([[1, 0, 0], [0, -1, 0], [0, 0, -1]], dtype=np.float32)
    pos_corrected_gl = cv_to_gl @ pos_corrected_cv
    R_corrected_gl = cv_to_gl @ R_corrected_cv @ cv_to_gl.T
    
    # Quaternion으로 변환
    quat_corrected = Rotation.from_matrix(R_corrected_gl).as_quat()
    
    camera_positions.append(pos_corrected_gl)
    
    # 보정된 pose로 이미지 중심을 통과하는 레이 계산
    ray_origin, ray_dir = get_image_center_ray(pos_corrected_gl, quat_corrected, fx, fy, cx, cy)
    ray_origins.append(ray_origin)
    ray_directions.append(ray_dir)

camera_positions = np.array(camera_positions)
camera_positions_original = np.array(camera_positions_original)
ray_origins = np.array(ray_origins)
ray_directions = np.array(ray_directions)

print(f"앵커 보정 완료")
print(f"보정 전 위치 범위:")
print(f"  X: {camera_positions_original[:, 0].min():.3f} ~ {camera_positions_original[:, 0].max():.3f}")
print(f"  Y: {camera_positions_original[:, 1].min():.3f} ~ {camera_positions_original[:, 1].max():.3f}")
print(f"  Z: {camera_positions_original[:, 2].min():.3f} ~ {camera_positions_original[:, 2].max():.3f}")
print(f"보정 후 위치 범위:")
print(f"  X: {camera_positions[:, 0].min():.3f} ~ {camera_positions[:, 0].max():.3f}")
print(f"  Y: {camera_positions[:, 1].min():.3f} ~ {camera_positions[:, 1].max():.3f}")
print(f"  Z: {camera_positions[:, 2].min():.3f} ~ {camera_positions[:, 2].max():.3f}")

# ===== 레이 교차점 계산 (Least Squares) =====
ray_intersection, avg_error, individual_distances, closest_points = compute_ray_intersection(ray_origins, ray_directions)

camera_center = camera_positions.mean(axis=0)
distance_from_camera = np.linalg.norm(ray_intersection - camera_center)

print(f"\n=== 레이 교차점 (Least Squares) ===")
print(f"교차점 좌표: ({ray_intersection[0]:.3f}, {ray_intersection[1]:.3f}, {ray_intersection[2]:.3f})")
print(f"평균 거리 오차: {avg_error*1000:.2f}mm")
print(f"최대 거리 오차: {max(individual_distances)*1000:.2f}mm")
print(f"최소 거리 오차: {min(individual_distances)*1000:.2f}mm")
print(f"카메라 중심에서 거리: {distance_from_camera:.3f}m")
print(f"\n카메라 위치 범위:")
print(f"  X: {camera_positions[:, 0].min():.3f} ~ {camera_positions[:, 0].max():.3f}")
print(f"  Y: {camera_positions[:, 1].min():.3f} ~ {camera_positions[:, 1].max():.3f}")
print(f"  Z: {camera_positions[:, 2].min():.3f} ~ {camera_positions[:, 2].max():.3f}")
print(f"카메라 위치 중심: ({camera_center[0]:.3f}, {camera_center[1]:.3f}, {camera_center[2]:.3f})")

# ===== 인터랙티브 3D 시각화 =====
fig = go.Figure()

# 카메라 위치
fig.add_trace(go.Scatter3d(
    x=camera_positions[:, 0],
    y=camera_positions[:, 1],
    z=camera_positions[:, 2],
    mode='markers',
    marker=dict(size=5, color='blue', opacity=0.6),
    name='Camera positions'
))

# 카메라 중심
fig.add_trace(go.Scatter3d(
    x=[camera_center[0]],
    y=[camera_center[1]],
    z=[camera_center[2]],
    mode='markers',
    marker=dict(size=10, color='cyan', symbol='square'),
    name='Camera center'
))

# 레이 시각화
ray_length = 2.0  # 레이를 2m까지 그리기
for i, (origin, direction) in enumerate(zip(ray_origins, ray_directions)):
    ray_end = origin + direction * ray_length
    error = individual_distances[i]
    # 거리 오차에 따라 색상 변경 (0-50mm: 초록, 50mm 이상: 빨강)
    color_intensity = min(1.0, error * 1000 / 50.0)
    if color_intensity < 0.5:
        color = f'rgb({int(255 * color_intensity * 2)}, 255, 0)'  # 초록 → 노랑
    else:
        color = f'rgb(255, {int(255 * (1 - color_intensity) * 2)}, 0)'  # 노랑 → 빨강
    
    fig.add_trace(go.Scatter3d(
        x=[origin[0], ray_end[0]],
        y=[origin[1], ray_end[1]],
        z=[origin[2], ray_end[2]],
        mode='lines',
        line=dict(color=color, width=2),
        showlegend=False,
        hoverinfo='skip'
    ))

# 교차점
fig.add_trace(go.Scatter3d(
    x=[ray_intersection[0]],
    y=[ray_intersection[1]],
    z=[ray_intersection[2]],
    mode='markers',
    marker=dict(size=15, color='red', symbol='diamond'),
    name=f'Ray intersection (error: {avg_error*1000:.1f}mm)'
))

# 각 레이에서 가장 가까운 점
fig.add_trace(go.Scatter3d(
    x=closest_points[:, 0],
    y=closest_points[:, 1],
    z=closest_points[:, 2],
    mode='markers',
    marker=dict(size=3, color='orange', opacity=0.5),
    name='Closest points on rays'
))

# 레이아웃 설정
# X, Y, Z 스케일 맞추기
all_points = np.vstack([camera_positions, ray_intersection.reshape(1, -1), closest_points])
x_range = [all_points[:, 0].min(), all_points[:, 0].max()]
y_range = [all_points[:, 1].min(), all_points[:, 1].max()]
z_range = [all_points[:, 2].min(), all_points[:, 2].max()]

max_range = max(
    x_range[1] - x_range[0],
    y_range[1] - y_range[0],
    z_range[1] - z_range[0]
) / 2.0

x_mid = (x_range[0] + x_range[1]) / 2.0
y_mid = (y_range[0] + y_range[1]) / 2.0
z_mid = (z_range[0] + z_range[1]) / 2.0

fig.update_layout(
    title='Ray Intersection Visualization (Interactive 3D)',
    scene=dict(
        xaxis_title='X (m)',
        yaxis_title='Y (m)',
        zaxis_title='Z (m)',
        aspectmode='cube',
        xaxis=dict(range=[x_mid - max_range, x_mid + max_range]),
        yaxis=dict(range=[y_mid - max_range, y_mid + max_range]),
        zaxis=dict(range=[z_mid - max_range, z_mid + max_range])
    ),
    width=1000,
    height=800
)

fig.show()

print(f"\n=== 최종 요약 ===")
print(f"카메라 위치 중심: ({camera_center[0]:.3f}, {camera_center[1]:.3f}, {camera_center[2]:.3f})")
print(f"오브젝트 추정 위치 (레이 교차점): ({ray_intersection[0]:.3f}, {ray_intersection[1]:.3f}, {ray_intersection[2]:.3f})")
print(f"카메라 중심에서 거리: {distance_from_camera:.3f}m")
print(f"평균 거리 오차: {avg_error*1000:.2f}mm")
print(f"거리 오차 범위: {min(individual_distances)*1000:.2f}mm ~ {max(individual_distances)*1000:.2f}mm")

총 프레임 수: 51
첫 프레임 앵커: pos=(0.422, -0.374, -0.711)
앵커 보정 적용 중...
앵커 보정 완료
보정 전 위치 범위:
  X: 0.070 ~ 0.711
  Y: -0.096 ~ 0.020
  Z: -0.703 ~ -0.264
보정 후 위치 범위:
  X: 0.057 ~ 0.698
  Y: -0.095 ~ 0.020
  Z: -0.703 ~ -0.267

=== 레이 교차점 (Least Squares) ===
교차점 좌표: (0.419, -0.519, -0.714)
평균 거리 오차: 17.46mm
최대 거리 오차: 57.52mm
최소 거리 오차: 1.72mm
카메라 중심에서 거리: 0.567m

카메라 위치 범위:
  X: 0.057 ~ 0.698
  Y: -0.095 ~ 0.020
  Z: -0.703 ~ -0.267
카메라 위치 중심: (0.400, -0.033, -0.424)



=== 최종 요약 ===
카메라 위치 중심: (0.400, -0.033, -0.424)
오브젝트 추정 위치 (레이 교차점): (0.419, -0.519, -0.714)
카메라 중심에서 거리: 0.567m
평균 거리 오차: 17.46mm
거리 오차 범위: 1.72mm ~ 57.52mm


In [55]:
# 메인 부피 계산 로직
# GLB 파일 로드
glb_path = '../../output/scene.glb'
points, scene_metadata = load_and_align_scene(glb_path)

if points is None:
    raise ValueError("포인트 클라우드를 로드할 수 없습니다.")

print(f"전체 포인트 수: {len(points):,}개")
print(f"Z 범위: {points[:, 2].min():.4f}m ~ {points[:, 2].max():.4f}m")

# 레이 교차점을 Scene 좌표계로 변환
first_frame_params = data[0]
ray_intersection_scene = transform_point_from_arcore(
    ray_intersection, 
    first_frame_params, 
    scene_metadata
)

print(f"\n레이 교차점 (Scene 좌표계): ({ray_intersection_scene[0]:.6f}, {ray_intersection_scene[1]:.6f}, {ray_intersection_scene[2]:.6f})")

# 1단계: 레이 교차점 아래 평면 검출 및 필터링
print(f"\n=== 1단계: 평면 검출 및 필터링 ===")
plane_z, plane_inlier_count = detect_plane_below_point(
    points, 
    ray_intersection_scene[2],
    search_below=PLANE_SEARCH_BELOW,
    thickness=PLANE_THICKNESS,
    min_points=PLANE_MIN_POINTS
)

if plane_z is not None:
    print(f"평면 검출: Z = {plane_z:.6f}m, 인라이너 수: {plane_inlier_count}개")
    # 평면 아래 포인트 제거
    points = points[points[:, 2] > plane_z]
    print(f"필터링 후 포인트 수: {len(points):,}개")
else:
    print("평면을 검출하지 못했습니다. 전체 포인트 사용")

# 전처리: 포인트를 Z축 기준으로 정렬만 수행 (필터링은 중심축 찾은 후에)
sorted_idx = np.argsort(points[:, 2])
sorted_points, sorted_heights = points[sorted_idx], points[sorted_idx, 2]

print(f"정렬 완료: {len(sorted_points):,}개 포인트")

# 2단계: 중심축 계산 (레이 교차점 기준 위아래 30cm, 5mm 간격)
print(f"\n=== 2단계: 중심축 계산 ===")
ray_z = ray_intersection_scene[2]
z_min_search = ray_z - AXIS_SEARCH_RANGE
z_max_search = ray_z + AXIS_SEARCH_RANGE

# 탐색 범위 내로 제한
z_min_search = max(z_min_search, sorted_heights.min())
z_max_search = min(z_max_search, sorted_heights.max())

z_range = np.arange(z_min_search, z_max_search + AXIS_SEARCH_STEP, AXIS_SEARCH_STEP)
print(f"탐색 범위: {z_min_search:.4f}m ~ {z_max_search:.4f}m ({len(z_range)}개 구간)")

circle_data = []
ray_intersection_2d = ray_intersection_scene[:2]  # XY 평면 투영

for z in z_range:
    start = np.searchsorted(sorted_heights, z - THICKNESS)
    end = np.searchsorted(sorted_heights, z + THICKNESS)
    if end - start < MIN_POINTS_PER_SLICE:
        continue
    
    # 전체 포인트로 slice 추출 (필터링 없음)
    slice_2d = sorted_points[start:end, :2]
    circle = fit_circle_ransac(
        slice_2d, 
        n_iter=50, 
        threshold=0.005, 
        min_inliers=10,
        ray_intersection_2d=ray_intersection_2d
    )
    
    if circle and 'ppm' in circle and 'angle_coverage' in circle and 'grid_coverage' in circle:
        circle_data.append({
            'z': z,
            'center': circle['center'],
            'radius': circle['radius'],
            'score': circle['score'],
            'ppm': circle['ppm'],
            'angle_coverage': circle['angle_coverage'],
            'grid_coverage': circle['grid_coverage'],
            'ray_distance': circle.get('ray_distance')
        })

print(f"원 검출 완료: {len(circle_data)}개 원 발견")

if not circle_data:
    print("컵 원을 찾지 못했습니다.")
else:
    # PPM 정규화
    all_ppms = [c['ppm'] for c in circle_data]
    ppm_min, ppm_max = min(all_ppms), max(all_ppms)
    
    # 각 원에 대해 최종 점수 계산
    for circle in circle_data:
        ppm_norm = (circle['ppm'] - ppm_min) / (ppm_max - ppm_min + 1e-8)
        inlier_ratio = circle['score']
        angle_coverage = circle['angle_coverage']
        grid_coverage_inverted = 1.0 - circle['grid_coverage']
        
        # 레이 거리 점수 (이미 0~1로 정규화됨)
        ray_distance_score = 1.0
        if circle['ray_distance'] is not None:
            ray_distance_score = max(0.0, 1.0 - circle['ray_distance'] / 0.1)
        
        # 최종 점수 계산 (가중치 적용)
        final_score = ((ppm_norm + 1e-8) ** WEIGHT_PPM_AXIS * 
                      (inlier_ratio + 1e-8) ** WEIGHT_INLIER_RATIO_AXIS * 
                      (angle_coverage + 1e-8) ** WEIGHT_ANGLE_COVERAGE_AXIS * 
                      (grid_coverage_inverted + 1e-8) ** WEIGHT_GRID_COVERAGE_AXIS *
                      (ray_distance_score + 1e-8) ** WEIGHT_RAY_DISTANCE_AXIS)
        
        circle['final_score'] = final_score
    
    # final_score 기준으로 정렬
    circle_data.sort(key=lambda x: x['final_score'], reverse=True)
    best_circle = circle_data[0]
    center_axis = best_circle['center']
    reference_z = best_circle['z']
    reference_radius = best_circle['radius']
    max_radius_limit = reference_radius * 1.15
    
    print(f"최종 선택된 중심축:")
    print(f"  중심축 좌표: ({center_axis[0]:.6f}, {center_axis[1]:.6f})")
    print(f"  기준 높이: {reference_z:.4f}m")
    print(f"  기준 반지름: {reference_radius:.4f}m")
    print(f"  최대 반지름 제한: {max_radius_limit:.4f}m")
    print(f"  최종 점수: {best_circle['final_score']:.6f}")
    if best_circle['ray_distance'] is not None:
        print(f"  레이 교차점 거리: {best_circle['ray_distance']*1000:.2f}mm")
    
    # 중심축 기준 필터링 (15cm 반지름)
    print(f"\n=== 중심축 기준 필터링 ===")
    FILTER_RADIUS = 0.15  # 15cm
    center_axis_distances_squared = np.sum((sorted_points[:, :2] - center_axis)**2, axis=1)
    axis_filter_mask = center_axis_distances_squared < FILTER_RADIUS**2
    filtered_points = sorted_points[axis_filter_mask]
    print(f"필터링 전: {len(sorted_points):,}개 포인트")
    print(f"필터링 후: {len(filtered_points):,}개 포인트 (중심축에서 {FILTER_RADIUS*100:.0f}cm 이내)")
    
    # 3단계: 부피 계산을 위한 원 검출
    print(f"\n=== 3단계: 부피 계산을 위한 원 검출 ===")
    volume_data = []
    z_min, z_max = filtered_points[:, 2].min(), filtered_points[:, 2].max()
    STEP = 0.002  # 2mm 간격
    
    # 기준 높이에서 원 검출
    ref_slice = get_slice(filtered_points, reference_z)
    if len(ref_slice) >= MIN_POINTS_PER_SLICE:
        ref_radius = detect_circle_at_axis(ref_slice[:, :2], center_axis, max_radius=max_radius_limit, prefer_inner=False)
        if ref_radius:
            add_volume_data(volume_data, reference_z, ref_radius)
            print(f"기준 높이({reference_z:.4f}m)에서 원 검출: 반지름={ref_radius:.4f}m")
    
    # 위쪽/아래쪽 탐색
    z_above = np.arange(reference_z + STEP, z_max + STEP, STEP)
    z_below = np.arange(reference_z - STEP, z_min - STEP, -STEP)
    print(f"위쪽 탐색: {len(z_above)}개 구간")
    print(f"아래쪽 탐색: {len(z_below)}개 구간")
    
    search_heights(filtered_points, z_above, center_axis, volume_data, max_radius=max_radius_limit)
    search_heights(filtered_points, z_below, center_axis, volume_data, max_radius=max_radius_limit)
    
    print(f"원 검출 완료: {len(volume_data)}개 단면 발견")
    
    
    
    # 5단계: 부피 계산
    print(f"\n=== 5단계: 부피 계산 ===")
    if volume_data:
        volume_data.sort(key=lambda x: x['z'])
        total_volume = sum((volume_data[i]['area'] + volume_data[i+1]['area']) / 2 * 
                          (volume_data[i+1]['z'] - volume_data[i]['z']) 
                          for i in range(len(volume_data) - 1))
        
        print(f"단면 개수: {len(volume_data)}개")
        print(f"높이: {volume_data[0]['z']:.4f}m ~ {volume_data[-1]['z']:.4f}m")
        print(f"부피: {total_volume * 1000:.2f} mL ({total_volume * 1e6:.2f} cm³)")
    else:
        print("원을 찾지 못했습니다.")

전체 포인트 수: 1,000,000개
Z 범위: -0.7665m ~ 0.3781m

레이 교차점 (Scene 좌표계): (0.260933, 0.085976, 0.084636)

=== 1단계: 평면 검출 및 필터링 ===
평면 검출: Z = -0.014976m, 인라이너 수: 9721개
필터링 후 포인트 수: 655,966개
정렬 완료: 655,966개 포인트 (slice 단위 필터링 적용 예정)

=== 2단계: 중심축 계산 ===
탐색 범위: -0.0150m ~ 0.3781m (80개 구간)
원 검출 완료: 51개 원 발견
최종 선택된 중심축:
  중심축 좌표: (0.250045, 0.080219)
  기준 높이: 0.2150m
  기준 반지름: 0.0420m
  최대 반지름 제한: 0.0482m
  최종 점수: 0.834906
  레이 교차점 거리: 12.32mm

=== 3단계: 부피 계산을 위한 원 검출 ===
기준 높이(0.2150m)에서 원 검출: 반지름=0.0413m
위쪽 탐색: 82개 구간
아래쪽 탐색: 116개 구간
원 검출 완료: 127개 단면 발견

=== 5단계: 부피 계산 ===
단면 개수: 127개
높이: -0.0150m ~ 0.2370m
부피: 1.14 mL (1144.39 cm³)
